In [1]:
import numpy as np

import tsplib95
import networkx as nx

import os
import pandas as pd
import gilsrvnd
import DBMEA
import grasp

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from networkx.algorithms.approximation import christofides

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

In [2]:
# get all files in Instances folder
files = os.listdir("newdifficult")
files

['DBMEA_21_4.tsp',
 'DBMEA_22_1.tsp',
 'DBMEA_22_15.tsp',
 'DBMEA_25_2.tsp',
 'DBMEA_26_12.tsp',
 'DBMEA_27_12.tsp',
 'DBMEA_28_12.tsp',
 'DBMEA_28_15.tsp',
 'DBMEA_29_7.tsp',
 'DBMEA_31_18.tsp',
 'DBMEA_31_7.tsp',
 'DBMEA_34_4.tsp',
 'DBMEA_41_12.tsp',
 'DBMEA_41_14.tsp',
 'DBMEA_41_6.tsp',
 'DBMEA_41_8.tsp',
 'DBMEA_42_9.tsp',
 'DBMEA_43_1.tsp',
 'DBMEA_43_2.tsp',
 'DBMEA_44_12.tsp',
 'DBMEA_46_3.tsp',
 'DBMEA_46_5.tsp',
 'DBMEA_49_14.tsp',
 'DBMEA_50_14.tsp',
 'DBMEA_50_9.tsp',
 'DBMEA_53_0.tsp',
 'DBMEA_54_7.tsp',
 'DBMEA_58_19.tsp',
 'DBMEA_61_8.tsp',
 'DBMEA_62_17.tsp',
 'DBMEA_63_4.tsp',
 'DBMEA_64_18.tsp',
 'DBMEA_68_13.tsp',
 'DBMEA_71_4.tsp',
 'DBMEA_73_19.tsp',
 'DBMEA_74_13.tsp',
 'DBMEA_75_10.tsp',
 'DBMEA_76_10.tsp',
 'DBMEA_77_9.tsp',
 'DBMEA_83_14.tsp',
 'DBMEA_85_0.tsp',
 'DBMEA_85_11.tsp',
 'DBMEA_85_6.tsp',
 'DBMEA_87_15.tsp',
 'DBMEA_91_4.tsp',
 'DBMEA_93_14.tsp',
 'DBMEA_93_4.tsp',
 'DBMEA_95_11.tsp',
 'DBMEA_96_15.tsp',
 'DBMEA_96_4.tsp',
 'GILS_20_2.tsp',
 'GILS_

In [3]:
# create dict for all the characterstics
characteristics = {"file":[], "n": [], "m": [], "ratio1": [], "ratio2": [], "density": [], "avg_clustering": [], "mst_size": [], "tsp_approx":[],
                   "edge_min": [], "edge_max": [], "edge_mean": [], "edge_std": [], "edge_var": [], "edge_q25": [], "edge_q75": [],
                   "between_min": [], "between_max": [], "between_mean": [], "between_std": [], "between_var": [], "between_q25": [], "between_q75": [],
                   "closeness_min": [], "closeness_max": [], "closeness_mean": [], "closeness_std": [], "closeness_var": [], "closeness_q25": [], "closeness_q75": []}

In [4]:
def tspCost(graph, dist_matrix):
    t = christofides(graph)
    # remove last index of t
    t = [x - 1 for x in t]
    t.pop()
    t = list(t)

    cost = sum((len(t) - i - 1) * dist_matrix[t[i], t[i + 1]] for i in range(len(t) - 1))
    return cost

In [6]:
for f in files:

    if f == "swiss42.tsp":
        continue

    problem = tsplib95.load('newdifficult/' + f)
    graph = problem.get_graph()
    dist_matrix = nx.to_numpy_matrix(graph)
    dist_array = nx.to_numpy_array(graph)

    n = len(graph.nodes)

    # get the number of edges
    m = len(graph.edges)

    # get the ratio of nodes to edges and vice versa
    ratio1 = n/m
    ratio2 = m/n
    # get the density of the graph
    density = nx.density(graph)

    # edge cost statistics
    edge_min = np.min(dist_matrix)
    edge_max = np.max(dist_matrix)
    edge_mean = np.mean(dist_matrix)
    edge_std = np.std(dist_matrix)
    edge_var = np.var(dist_matrix)
    edge_q25 = np.quantile(dist_array, 0.25)
    edge_q75 = np.quantile(dist_array, 0.75)

    # local clustering statistics
    avg_clustering = nx.average_clustering(graph, weight='weight')

    # minimum spanning tree size
    mst = nx.minimum_spanning_tree(graph, weight='weight')
    mst_size = len(mst.edges)
    tsp_approx = tspCost(graph, dist_matrix)

    # betweenness centrality statistics
    between = nx.betweenness_centrality(graph, weight='weight')
    between_min = np.min(list(between.values()))
    between_max = np.max(list(between.values()))
    between_mean = np.mean(list(between.values()))
    between_std = np.std(list(between.values()))
    between_var = np.var(list(between.values()))
    between_q25 = np.quantile(list(between.values()), 0.25)
    between_q75 = np.quantile(list(between.values()), 0.75)

    # closeness centrality statistics
    closeness = nx.closeness_centrality(graph, distance='weight')
    closeness_min = np.min(list(closeness.values()))
    closeness_max = np.max(list(closeness.values()))
    closeness_mean = np.mean(list(closeness.values()))
    closeness_std = np.std(list(closeness.values()))
    closeness_var = np.var(list(closeness.values()))
    closeness_q25 = np.quantile(list(closeness.values()), 0.25)
    closeness_q75 = np.quantile(list(closeness.values()), 0.75)

    # add information to the dictionary
    characteristics["file"].append(f)
    characteristics["n"].append(n)
    characteristics["m"].append(m)
    characteristics["ratio1"].append(ratio1)
    characteristics["ratio2"].append(ratio2)
    characteristics["density"].append(density)
    characteristics["avg_clustering"].append(avg_clustering)
    characteristics["mst_size"].append(mst_size)
    characteristics["tsp_approx"].append(tsp_approx)
    characteristics["edge_min"].append(edge_min)
    characteristics["edge_max"].append(edge_max)
    characteristics["edge_mean"].append(edge_mean)
    characteristics["edge_std"].append(edge_std)
    characteristics["edge_var"].append(edge_var)
    characteristics["edge_q25"].append(edge_q25)
    characteristics["edge_q75"].append(edge_q75)
    characteristics["between_min"].append(between_min)
    characteristics["between_max"].append(between_max)
    characteristics["between_mean"].append(between_mean)
    characteristics["between_std"].append(between_std)
    characteristics["between_var"].append(between_var)
    characteristics["between_q25"].append(between_q25)
    characteristics["between_q75"].append(between_q75)
    characteristics["closeness_min"].append(closeness_min)
    characteristics["closeness_max"].append(closeness_max)
    characteristics["closeness_mean"].append(closeness_mean)
    characteristics["closeness_std"].append(closeness_std)
    characteristics["closeness_var"].append(closeness_var)
    characteristics["closeness_q25"].append(closeness_q25)
    characteristics["closeness_q75"].append(closeness_q75)

    print(f'{f} done')



DBMEA_21_4.tsp done
DBMEA_22_1.tsp done
DBMEA_22_15.tsp done
DBMEA_25_2.tsp done
DBMEA_26_12.tsp done
DBMEA_27_12.tsp done
DBMEA_28_12.tsp done
DBMEA_28_15.tsp done
DBMEA_29_7.tsp done
DBMEA_31_18.tsp done
DBMEA_31_7.tsp done
DBMEA_34_4.tsp done
DBMEA_41_12.tsp done
DBMEA_41_14.tsp done
DBMEA_41_6.tsp done
DBMEA_41_8.tsp done
DBMEA_42_9.tsp done
DBMEA_43_1.tsp done
DBMEA_43_2.tsp done
DBMEA_44_12.tsp done
DBMEA_46_3.tsp done
DBMEA_46_5.tsp done
DBMEA_49_14.tsp done
DBMEA_50_14.tsp done
DBMEA_50_9.tsp done
DBMEA_53_0.tsp done
DBMEA_54_7.tsp done
DBMEA_58_19.tsp done
DBMEA_61_8.tsp done
DBMEA_62_17.tsp done
DBMEA_63_4.tsp done
DBMEA_64_18.tsp done
DBMEA_68_13.tsp done
DBMEA_71_4.tsp done
DBMEA_73_19.tsp done
DBMEA_74_13.tsp done
DBMEA_75_10.tsp done
DBMEA_76_10.tsp done
DBMEA_77_9.tsp done
DBMEA_83_14.tsp done
DBMEA_85_0.tsp done
DBMEA_85_11.tsp done
DBMEA_85_6.tsp done
DBMEA_87_15.tsp done
DBMEA_91_4.tsp done
DBMEA_93_14.tsp done
DBMEA_93_4.tsp done
DBMEA_95_11.tsp done
DBMEA_96_15.tsp 

In [7]:
chars = pd.DataFrame.from_dict(characteristics)
chars

,file,n,m,ratio1,ratio2,density,avg_clustering,mst_size,tsp_approx,edge_min,...,between_var,between_q25,between_q75,closeness_min,closeness_max,closeness_mean,closeness_std,closeness_var,closeness_q25,closeness_q75
0,DBMEA_21_4.tsp,21,231,0.090909,11.0,1.100000,0.410978,20,205335.0,0.0,...,0.000023,0.052632,0.057895,0.000307,0.000626,0.000470,0.000100,9.934298e-09,0.000377,0.000563
1,DBMEA_22_1.tsp,22,253,0.086957,11.5,1.095238,0.410231,21,240658.0,0.0,...,0.000033,0.050000,0.057540,0.000290,0.000585,0.000443,0.000081,6.631258e-09,0.000384,0.000526
2,DBMEA_22_15.tsp,22,253,0.086957,11.5,1.095238,0.367077,21,227910.0,0.0,...,0.000025,0.050000,0.057143,0.000289,0.000615,0.000460,0.000096,9.183057e-09,0.000394,0.000547
3,DBMEA_25_2.tsp,25,325,0.076923,13.0,1.083333,0.428445,24,264210.0,0.0,...,0.000030,0.043478,0.050725,0.000337,0.000581,0.000445,0.000072,5.237894e-09,0.000393,0.000508
4,DBMEA_26_12.tsp,26,351,0.074074,13.5,1.080000,0.441006,25,251036.0,0.0,...,0.000020,0.041667,0.048194,0.000312,0.000541,0.000416,0.000063,3.950803e-09,0.000361,0.000458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GILS_93_2.tsp,93,4371,0.021277,47.0,1.021739,0.393550,92,1896193.0,0.0,...,0.000007,0.011626,0.015944,0.000289,0.000511,0.000390,0.000060,3.608474e-09,0.000338,0.000436
96,GILS_95_16.tsp,95,4560,0.020833,48.0,1.021277,0.386529,94,1665710.0,0.0,...,0.000008,0.011674,0.016416,0.000274,0.000535,0.000401,0.000069,4.734754e-09,0.000349,0.000453
97,GILS_95_5.tsp,95,4560,0.020833,48.0,1.021277,0.376426,94,1938745.0,0.0,...,0.000007,0.011575,0.015873,0.000274,0.000505,0.000381,0.000060,3.574450e-09,0.000336,0.000421
98,GILS_97_12.tsp,97,4753,0.020408,49.0,1.020833,0.386491,96,1959628.0,0.0,...,0.000007,0.011404,0.015399,0.000264,0.000519,0.000392,0.000064,4.089912e-09,0.000340,0.000435


In [8]:
# save chars to csv
chars.to_csv('newdiff.csv', index=False)

In [14]:
# merge the two dataframes on filename
df2 = pd.merge(df, chars, on='file')
df2

,file,GILS_cost,DBMEA_cost,GRASP_cost,GILS_time,DBMEA_time,GRASP_time,min_method,n,m,...,between_var,between_q25,between_q75,closeness_min,closeness_max,closeness_mean,closeness_std,closeness_var,closeness_q25,closeness_q75
0,att48.tsp,226229.0,207771.0,301702.0,6.474685,5.288543,0.026019,DBMEA_cost,48,1176,...,0.000008,0.022051,0.026965,0.000574,0.001325,0.001012,0.000205,4.221718e-08,0.000889,0.001191
1,berlin52.tsp,145419.0,143278.0,191072.0,6.054587,5.374160,0.031533,DBMEA_cost,52,1378,...,0.000065,0.021686,0.032662,0.000919,0.002555,0.001893,0.000507,2.566472e-07,0.001447,0.002371
2,brazil58.tsp,532454.0,535150.0,709540.0,8.013889,9.045840,0.043413,GILS_cost,58,1711,...,0.000591,0.020050,0.042040,0.000201,0.000736,0.000562,0.000145,2.104825e-08,0.000458,0.000692
3,burma14.tsp,16457.0,16457.0,18393.0,0.156218,0.082072,0.001000,GRASP_time,14,105,...,0.000097,0.000000,0.006410,0.001313,0.002677,0.002193,0.000407,1.654824e-07,0.002048,0.002518
4,dantzig42.tsp,12392.0,12256.0,14523.0,2.579336,3.325098,0.016044,DBMEA_cost,42,903,...,0.000788,0.033226,0.062177,0.008369,0.019061,0.014587,0.003118,9.721282e-06,0.011758,0.016824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,line_90_5.tsp,55730.0,57558.0,157350.0,35.477098,36.326975,0.233857,GILS_cost,90,4095,...,0.000435,0.050520,0.072751,0.002192,0.004113,0.003227,0.000642,4.123292e-07,0.002620,0.003881
335,line_90_6.tsp,48884.0,45916.0,128976.0,30.191444,37.348990,0.232372,DBMEA_cost,90,4095,...,0.000730,0.049839,0.071527,0.002220,0.004410,0.003481,0.000709,5.026715e-07,0.002873,0.004060
336,line_90_7.tsp,51464.0,49038.0,124368.0,28.821121,44.483338,0.232510,DBMEA_cost,90,4095,...,0.000393,0.048856,0.069497,0.002238,0.005424,0.004240,0.000992,9.846681e-07,0.003531,0.005157
337,line_90_8.tsp,60845.0,65825.0,123373.0,28.282006,39.745050,0.231058,GILS_cost,90,4095,...,0.000680,0.049190,0.072960,0.002171,0.004137,0.003256,0.000649,4.205608e-07,0.002625,0.003914


In [15]:
df2.min_method.value_counts()

min_method
DBMEA_cost    186
GILS_cost     120
GRASP_time     33
Name: count, dtype: int64

In [16]:
# save df2 to csv
df2.to_csv('matrix_final.csv', index=False)

In [17]:
# train a classifier to predict the min_method


In [18]:
# get the data
df2 = pd.read_csv('matrix_final.csv')
df2

,file,GILS_cost,DBMEA_cost,GRASP_cost,GILS_time,DBMEA_time,GRASP_time,min_method,n,m,...,between_var,between_q25,between_q75,closeness_min,closeness_max,closeness_mean,closeness_std,closeness_var,closeness_q25,closeness_q75
0,att48.tsp,226229.0,207771.0,301702.0,6.474685,5.288543,0.026019,DBMEA_cost,48,1176,...,0.000008,0.022051,0.026965,0.000574,0.001325,0.001012,0.000205,4.221718e-08,0.000889,0.001191
1,berlin52.tsp,145419.0,143278.0,191072.0,6.054587,5.374160,0.031533,DBMEA_cost,52,1378,...,0.000065,0.021686,0.032662,0.000919,0.002555,0.001893,0.000507,2.566472e-07,0.001447,0.002371
2,brazil58.tsp,532454.0,535150.0,709540.0,8.013889,9.045840,0.043413,GILS_cost,58,1711,...,0.000591,0.020050,0.042040,0.000201,0.000736,0.000562,0.000145,2.104825e-08,0.000458,0.000692
3,burma14.tsp,16457.0,16457.0,18393.0,0.156218,0.082072,0.001000,GRASP_time,14,105,...,0.000097,0.000000,0.006410,0.001313,0.002677,0.002193,0.000407,1.654824e-07,0.002048,0.002518
4,dantzig42.tsp,12392.0,12256.0,14523.0,2.579336,3.325098,0.016044,DBMEA_cost,42,903,...,0.000788,0.033226,0.062177,0.008369,0.019061,0.014587,0.003118,9.721282e-06,0.011758,0.016824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,line_90_5.tsp,55730.0,57558.0,157350.0,35.477098,36.326975,0.233857,GILS_cost,90,4095,...,0.000435,0.050520,0.072751,0.002192,0.004113,0.003227,0.000642,4.123292e-07,0.002620,0.003881
335,line_90_6.tsp,48884.0,45916.0,128976.0,30.191444,37.348990,0.232372,DBMEA_cost,90,4095,...,0.000730,0.049839,0.071527,0.002220,0.004410,0.003481,0.000709,5.026715e-07,0.002873,0.004060
336,line_90_7.tsp,51464.0,49038.0,124368.0,28.821121,44.483338,0.232510,DBMEA_cost,90,4095,...,0.000393,0.048856,0.069497,0.002238,0.005424,0.004240,0.000992,9.846681e-07,0.003531,0.005157
337,line_90_8.tsp,60845.0,65825.0,123373.0,28.282006,39.745050,0.231058,GILS_cost,90,4095,...,0.000680,0.049190,0.072960,0.002171,0.004137,0.003256,0.000649,4.205608e-07,0.002625,0.003914


In [19]:
# get the features
X = df2.drop(['file', 'min_method', "GILS_cost", "GILS_time", "GRASP_cost", "GRASP_time", "DBMEA_cost", "DBMEA_time"], axis=1)
y = df2['min_method']


In [21]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [22]:
# train the classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


RandomForestClassifier()

In [23]:
# get the predictions
y_pred = clf.predict(X_test)


In [24]:
# get the accuracy
accuracy_score(y_test, y_pred)

0.5980392156862745

In [25]:
# get the confusion matrix
confusion_matrix(y_test, y_pred)

array([[41, 21,  2],
       [13, 14,  2],
       [ 1,  2,  6]], dtype=int64)

In [26]:
# get the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  DBMEA_cost       0.75      0.64      0.69        64
   GILS_cost       0.38      0.48      0.42        29
  GRASP_time       0.60      0.67      0.63         9

    accuracy                           0.60       102
   macro avg       0.57      0.60      0.58       102
weighted avg       0.63      0.60      0.61       102



In [31]:
# rank the features by importance
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
# plot the features
plt.barh(feature_importances.index, feature_importances.importance)
plt.show()


,importance
between_q25,0.061629
tsp_approx,0.061198
between_mean,0.058622
between_q75,0.057403
avg_clustering,0.049619
density,0.048625
between_max,0.039240
between_std,0.036947
m,0.035831
ratio2,0.034066
